<a href="https://colab.research.google.com/github/mohamedalaaaz/testpytroch/blob/main/Airplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell

cat > main.cpp <<EOF
#include <torch/torch.h>
#include <torchvision/models/resnet.h>
#include <iostream>
#include <iomanip>

// Simple binary wrapper: 1 if airplane (class=0), else 0
struct BinaryAirplaneDataset : torch::data::datasets::Dataset<BinaryAirplaneDataset> {
    torch::data::datasets::CIFAR10 cifar;
    BinaryAirplaneDataset(const std::string& root, bool train, torch::data::transforms::TensorTransform transform)
        : cifar(root, train, transform) {}

    torch::data::Example<> get(size_t idx) override {
        auto example = cifar.get(idx);
        auto label = example.target.item<int64_t>();
        int64_t y = (label == 0) ? 1 : 0;
        return {example.data, torch::tensor(y, torch::kLong)};
    }
    torch::optional<size_t> size() const override { return cifar.size(); }
};

int main() {
    torch::manual_seed(42);
    torch::Device device(torch::cuda::is_available() ? torch::kCUDA : torch::kCPU);
    std::cout << "Using device: " << (device.is_cuda() ? "CUDA" : "CPU") << "\n";

    const int batch_size = 128;
    const int epochs = 5;
    const int image_size = 224;

    // Transforms
    auto transform = torch::data::transforms::Compose({
        torch::data::transforms::Resize({image_size, image_size}),
        torch::data::transforms::ToTensor(),
        torch::data::transforms::Normalize({0.485,0.456,0.406}, {0.229,0.224,0.225})
    });

    // Datasets + loaders
    auto train_ds = BinaryAirplaneDataset("./data", true, transform).map(torch::data::transforms::Stack<>());
    auto test_ds  = BinaryAirplaneDataset("./data", false, transform).map(torch::data::transforms::Stack<>());

    auto train_loader = torch::data::make_data_loader(std::move(train_ds), batch_size);
    auto test_loader  = torch::data::make_data_loader(std::move(test_ds), batch_size);

    // Model: ResNet18
    auto model = vision::models::ResNet18(/*num_classes=*/1);
    model->to(device);

    torch::optim::Adam optimizer(model->parameters(), torch::optim::AdamOptions(1e-4));

    // Training loop
    for (int epoch = 1; epoch <= epochs; epoch++) {
        model->train();
        size_t batch_idx = 0;
        double total_loss = 0.0;

        for (auto& batch : *train_loader) {
            auto data = batch.data.to(device);
            auto targets = batch.target.to(device).to(torch::kFloat32);

            optimizer.zero_grad();
            auto logits = model->forward(data).view(-1);
            auto loss = torch::binary_cross_entropy_with_logits(logits, targets);
            loss.backward();
            optimizer.step();

            total_loss += loss.item<double>();
            if (batch_idx++ % 50 == 0)
                std::cout << "Epoch " << epoch << " [" << batch_idx << "] Loss: " << loss.item<double>() << "\n";
        }
        std::cout << "Epoch " << epoch << " Avg Loss: " << total_loss / batch_idx << "\n";

        // Evaluation
        model->eval();
        int correct = 0, total = 0;
        torch::NoGradGuard no_grad;
        for (auto& batch : *test_loader) {
            auto data = batch.data.to(device);
            auto targets = batch.target.to(device);

            auto logits = model->forward(data).view(-1).sigmoid();
            auto preds = (logits >= 0.5).to(torch::kLong);
            correct += preds.eq(targets).sum().item<int64_t>();
            total += targets.size(0);
        }
        std::cout << "Test Accuracy: " << std::fixed << std::setprecision(4)
                  << static_cast<double>(correct) / total << "\n";
    }

    torch::save(model, "airplane_model.pt");
    std::cout << "Model saved to airplane_model.pt\n";
}
EOF

g++ main.cpp -o main $(pkg-config --cflags --libs libtorch) -ltorchvision -std=c++17
./main

/bin/bash: -c: line 7: syntax error near unexpected token `('
/bin/bash: -c: line 7: `// Simple binary wrapper: 1 if airplane (class=0), else 0'


CalledProcessError: Command '
#include <torch/torch.h>
#include <torchvision/models/resnet.h>
#include <iostream>
#include <iomanip>

// Simple binary wrapper: 1 if airplane (class=0), else 0
struct BinaryAirplaneDataset : torch::data::datasets::Dataset<BinaryAirplaneDataset> {
    torch::data::datasets::CIFAR10 cifar;
    BinaryAirplaneDataset(const std::string& root, bool train, torch::data::transforms::TensorTransform transform)
        : cifar(root, train, transform) {}

    torch::data::Example<> get(size_t idx) override {
        auto example = cifar.get(idx);
        auto label = example.target.item<int64_t>();
        int64_t y = (label == 0) ? 1 : 0;
        return {example.data, torch::tensor(y, torch::kLong)};
    }
    torch::optional<size_t> size() const override { return cifar.size(); }
};

int main() {
    torch::manual_seed(42);
    torch::Device device(torch::cuda::is_available() ? torch::kCUDA : torch::kCPU);
    std::cout << "Using device: " << (device.is_cuda() ? "C